# Solve EEG parametric forward problem

In this example, we go through the process of solving EEG parametric forward problem for the model generated in [this example](1_model_creation.ipynb). The result of this process is a `EEGParametricForwardSolution` which can be used to generate any `EEGForwardSolution` or leadfield matrix based on a set of parameters.

## Model loading

As for [this example](2_eeg_forward.ipynb), we first have to load the previously generated `FEModel`.

In [1]:
from pathlib import Path
from shamo import FEModel

model_name = "model"
model_path = Path(("./derivatives/1_model_creation/{name}"
                   "/{name}.json").format(name=model_name))
model = FEModel.load(model_path)

The model can now be used to generate the parametric leadfield matrix.

## Problem definition

To generate an `EEGParametricForwardSolution`, we must create an `EEGParametricForwardProblem`. The exact same steps are used to define this problem:

In [2]:
from shamo import EEGParametricForwardProblem

problem = EEGParametricForwardProblem()

# Set reference
problem.set_reference("A")

# Add markers
problem.add_markers(["C", "G"])

# Add regions of interest
problem.add_regions_of_interest(["b", "c"])

{'regions_of_interest': ['b', 'c'],
 'markers': ['C', 'G'],
 'electrical_conductivity': {},
 'reference': 'A'}

The only thing that changes here is how we define the electrical conductivity of the tissues. Indeed, as we want them to be parametric, we have to provide the problem with distributions for their values. The available distributions are:
- `ConstantDistribution`: It defines a fixed variable and only takes one parameter which is the value.
- `TruncatedNormalDistribution`: It defines a bounded normal distribution.
- `UniformDistribution`: It defines a uniform distribution and takes two parameters (its min and max values).

As for the `EEGForwardProblem`, we can specify an anisotropic field for each tissue.

In [3]:
from shamo import ConstantDistribution, UniformDistribution

problem.set_electrical_conductivities({"a": ConstantDistribution(1.0), "c": UniformDistribution(0.25, 0.75)})
problem.set_electrical_conductivity("b", UniformDistribution(0.1, 0.9), "b_anisotropy")

{'regions_of_interest': ['b', 'c'],
 'markers': ['C', 'G'],
 'electrical_conductivity': {'a': {'value': {'name': 'constant', 'value': 1.0},
   'anisotropy': ''},
  'c': {'value': {'name': 'uniform', 'minimum': 0.25, 'maximum': 0.75},
   'anisotropy': ''},
  'b': {'value': {'name': 'uniform', 'minimum': 0.1, 'maximum': 0.9},
   'anisotropy': 'b_anisotropy'}},
 'reference': 'A'}

## Problem resolution

Finally, we can solve the problem. This solver can accept more arguments. For instance, we can provide a `method` argument which defines how we want to solve the problem:
- `METHOD_SEQ` to use only one core.
- `METHOD_MULTI` to use multiple cores on a single machine.
- `METHOD_MPI` to use multiple cores on a distributed system.
- `METHOD_JOBS` to generate one python script by sub-problem. Those scripts can then be run to generate separate `EEGForwardSolution`. When using this method, you must call the `finalize()` method after loading the solution to allow it to discover the sub-solutions and generate the surrogate model.

In [4]:
solution = problem.solve("parametric_solution", 
                         "./derivatives/4_eeg_parametric_forward", model,
                         method=EEGParametricForwardProblem.METHOD_MULTI,
                         n_evals=10)

## Surrogate model usage

Once the `EEGParametricForwardSolution` is properly generated and that `finalize()` have been called, we can use it to produce arbitrary `EEGForwardSolution` and leadfield matrix.

For instance, we can create a solution which has a certain set of electrical conductivities by using the `generate_solution()` method.

In [5]:
arbitrary_solution = solution.generate_solution("solution", "./derivatives/4_eeg_parametric_forward", 
                                                c=0.35, b=0.62)

The resulting `EEGForwardSolution` can be used just like any other to evaluate a source vector or for further computations.

It is also possible to generate only the matrix by using the `generate_matrix()` method.

In [6]:
arbitrary_matrix = solution.generate_matrix(c=0.35, b=0.62)